In [29]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.base import clone
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

%matplotlib inline

Importar y revisar dataset

In [30]:
# importar dataset

!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00603/in-vehicle-coupon-recommendation.csv

--2021-06-28 21:52:07--  https://archive.ics.uci.edu/ml/machine-learning-databases/00603/in-vehicle-coupon-recommendation.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2163456 (2.1M) [application/x-httpd-php]
Saving to: ‘in-vehicle-coupon-recommendation.csv.1’

in-vehicle-coupon-r 100%[===================>]   2.06M  4.31MB/s    in 0.5s    

2021-06-28 21:52:08 (4.31 MB/s) - ‘in-vehicle-coupon-recommendation.csv.1’ saved [2163456/2163456]



In [31]:
%ls

categorical_columns.pkl
dictionary_of_columns_with_categorical_to_index.pkl
dictionary_of_columns_with_index_to_categorical.pkl
in-vehicle-coupon-recommendation.csv
in-vehicle-coupon-recommendation.csv.1
numeric_columns.pkl
preprocessed_dataset_titanic_train.pkl
sample_data/


In [32]:
#Lectura del dataset
file_name = "in-vehicle-coupon-recommendation.csv"

sheet_name = "in-vehicle-coupon-recommendation"

dataset = pd.read_csv(file_name, sheet_name, delimiter=',')

# Visualizacion del Dataset

In [33]:
#Tamaño del dataset
dataset.shape

(12684, 26)

In [34]:
#Tamaño del dataset
dataset.head()

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,has_children,education,occupation,income,car,Bar,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,No Urgent Place,Alone,Sunny,55,2PM,Restaurant(<20),1d,Female,21,Unmarried partner,1,Some college - no degree,Unemployed,$37500 - $49999,NaN,never,never,NaN,4~8,1~3,1,0,0,0,1,1
1,No Urgent Place,Friend(s),Sunny,80,10AM,Coffee House,2h,Female,21,Unmarried partner,1,Some college - no degree,Unemployed,$37500 - $49999,NaN,never,never,NaN,4~8,1~3,1,0,0,0,1,0
2,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Female,21,Unmarried partner,1,Some college - no degree,Unemployed,$37500 - $49999,NaN,never,never,NaN,4~8,1~3,1,1,0,0,1,1
3,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,2h,Female,21,Unmarried partner,1,Some college - no degree,Unemployed,$37500 - $49999,NaN,never,never,NaN,4~8,1~3,1,1,0,0,1,0
4,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,1d,Female,21,Unmarried partner,1,Some college - no degree,Unemployed,$37500 - $49999,NaN,never,never,NaN,4~8,1~3,1,1,0,0,1,0


In [35]:
dataset.dtypes

destination             object
passanger               object
weather                 object
temperature              int64
time                    object
coupon                  object
expiration              object
gender                  object
age                     object
maritalStatus           object
has_children             int64
education               object
occupation              object
income                  object
car                     object
Bar                     object
CoffeeHouse             object
CarryAway               object
RestaurantLessThan20    object
Restaurant20To50        object
toCoupon_GEQ5min         int64
toCoupon_GEQ15min        int64
toCoupon_GEQ25min        int64
direction_same           int64
direction_opp            int64
Y                        int64
dtype: object

In [36]:
(dataset.Y==1).sum()

7210

# Preprocesamiento del Dataset

In [37]:
# Se visualizan los valores unicos en cada una de las columnas con el fin de verificar concordacion con el dataset.
for col in dataset:
    print(col, dataset[col].unique())

destination ['No Urgent Place' 'Home' 'Work']
passanger ['Alone' 'Friend(s)' 'Kid(s)' 'Partner']
weather ['Sunny' 'Rainy' 'Snowy']
temperature [55 80 30]
time ['2PM' '10AM' '6PM' '7AM' '10PM']
coupon ['Restaurant(<20)' 'Coffee House' 'Carry out & Take away' 'Bar'
 'Restaurant(20-50)']
expiration ['1d' '2h']
gender ['Female' 'Male']
age ['21' '46' '26' '31' '41' '50plus' '36' 'below21']
maritalStatus ['Unmarried partner' 'Single' 'Married partner' 'Divorced' 'Widowed']
has_children [1 0]
education ['Some college - no degree' 'Bachelors degree' 'Associates degree'
 'High School Graduate' 'Graduate degree (Masters or Doctorate)'
 'Some High School']
occupation ['Unemployed' 'Architecture & Engineering' 'Student'
 'Education&Training&Library' 'Healthcare Support'
 'Healthcare Practitioners & Technical' 'Sales & Related' 'Management'
 'Arts Design Entertainment Sports & Media' 'Computer & Mathematical'
 'Life Physical Social Science' 'Personal Care & Service'
 'Community & Social Services' 

De los datos anteriores se puede observar que existen dos caracteristicas (Car y toCoupon_GEQ5min) que no se encuentran dentro de la lista de caracteristicas del dataset original (https://archive.ics.uci.edu/ml/datasets/in-vehicle+coupon+recommendation)

In [38]:
# Numero de valores unicos en las columnas
dataset.nunique()

destination              3
passanger                4
weather                  3
temperature              3
time                     5
coupon                   5
expiration               2
gender                   2
age                      8
maritalStatus            5
has_children             2
education                6
occupation              25
income                   9
car                      5
Bar                      5
CoffeeHouse              5
CarryAway                5
RestaurantLessThan20     5
Restaurant20To50         5
toCoupon_GEQ5min         1
toCoupon_GEQ15min        2
toCoupon_GEQ25min        2
direction_same           2
direction_opp            2
Y                        2
dtype: int64

Viendo el numero de valores unicos en las columnas del dataset se puede notar que la caracteristica "toCoupon_GEQ5min" no aportara nada a la prediccion ya que solamente existe un valor unico para esta caracteristica. Por lo tanto el algoritmo tendria que adivinar que predecir en casos donde dicha caracteristica presente un valor distinto.

In [39]:
# Porcetaje de filas (registros) vacias por caracteristica 
dataset.isna().sum() / len(dataset)

destination             0.000000
passanger               0.000000
weather                 0.000000
temperature             0.000000
time                    0.000000
coupon                  0.000000
expiration              0.000000
gender                  0.000000
age                     0.000000
maritalStatus           0.000000
has_children            0.000000
education               0.000000
occupation              0.000000
income                  0.000000
car                     0.991485
Bar                     0.008436
CoffeeHouse             0.017108
CarryAway               0.011905
RestaurantLessThan20    0.010249
Restaurant20To50        0.014901
toCoupon_GEQ5min        0.000000
toCoupon_GEQ15min       0.000000
toCoupon_GEQ25min       0.000000
direction_same          0.000000
direction_opp           0.000000
Y                       0.000000
dtype: float64

Por otro lado, al observar el porcetaje de filas (registros) vacias por caracteristica se puede notar que se tiene un valor de casi el 100% en la caracteristica "car". En este caso, seria conveniente descartar la columna ya que rellenar tanta informacion con valores como la media, mediana o moda no generaria informacion relevante o representativa. 

Cabe resaltar que a diferencia de la caracteristica "car", el numero de nulos en las caracteristicas "Bar", "CoffeHouse", "CarryAway", "RestaurantLessThan20" y "Restaurant20To50" es tan pequeño (maximo 1.7%) que los registros asociados a ellos pueden ser descartados. (opcional) Sin embargo, segun la fuente de la base de datos estan permitidos los nan en estas caracteristica a excepcion de "RestaurantLessThan20". Por lo tanto solo se descartarian las filas con nulos en esta caracteristica.


In [40]:
# Funcion para el preprocesamiento de los datos
def preprocess_dataset(dataset, save_metadata=True):
  preprocessed_dataset = dataset.copy()

  #Determinamos aquellas variables que tengan un porcentaje de registros únicos por cada variable mayor al valor de 0.7
  threshold = 0.7

  unique_percentages = dataset.nunique() / len(dataset)

  criteria = unique_percentages > threshold

  columns_to_filter = unique_percentages[criteria].keys()

  #Eliminamos las variables seleccionadas en el paso anterior
  preprocessed_dataset.drop(columns_to_filter, axis=1, inplace=True)

  #Eliminamos las columnas "toCoupon_GEQ5min" y "car"
  preprocessed_dataset.drop("toCoupon_GEQ5min", axis=1, inplace=True)
  preprocessed_dataset.drop("car", axis=1, inplace=True)

  #Eliminamos los registros duplicados
  preprocessed_dataset.drop_duplicates(keep="first", inplace=True)

  #Determinamos que variables son del tipo númerico y cuales son categóricas
  numeric_columns = list()
  categorical_columns = list()
  dictionary_of_columns_with_index_to_categorical = dict()
  dictionary_of_columns_with_categorical_to_index = dict()

  for column in preprocessed_dataset:
    #Determinamos si la variable es numérica o no
    if pd.api.types.is_numeric_dtype(preprocessed_dataset[column]):
      numeric_columns.append(column)
    else:
      #Modificamos el tipo de dato de la variable mediante "astype"
      preprocessed_dataset[column] = preprocessed_dataset[column].astype("category")

      #Verificamos si el tipo de dato de la variable fue transformado a categórico correctamente
      if not pd.api.types.is_categorical_dtype(preprocessed_dataset[column]):
        raise Exception("La columna {} no se transformó correctamente a categórica".format(column))

      dictionary_of_columns_with_index_to_categorical[column] = dict()
      dictionary_of_columns_with_categorical_to_index[column] = dict()
      
      #Indexamos los valores (categorías), sin tomar en consideración los nulos, de la variable y guardamos esa información en los diccionarios
      for index, category in enumerate(preprocessed_dataset[column].cat.categories):
        dictionary_of_columns_with_index_to_categorical[column][index] = category
        dictionary_of_columns_with_categorical_to_index[column][category] = index
      
      categorical_columns.append(column)

  #Descartamos las filas (registros) de "Bar"
  criteria = preprocessed_dataset.Bar.isna()
  preprocessed_dataset.drop(preprocessed_dataset[criteria].index, inplace=True)

  #Descartamos las filas (registros) de "CoffeeHouse"
  criteria = preprocessed_dataset.CoffeeHouse.isna()
  preprocessed_dataset.drop(preprocessed_dataset[criteria].index, inplace=True)

  #Descartamos las filas (registros) de "CarryAway"
  criteria = preprocessed_dataset.CarryAway.isna()
  preprocessed_dataset.drop(preprocessed_dataset[criteria].index, inplace=True)

  #Descartamos las filas (registros) de "RestaurantLessThan20"
  criteria = preprocessed_dataset.RestaurantLessThan20.isna()
  preprocessed_dataset.drop(preprocessed_dataset[criteria].index, inplace=True)

  #Descartamos las filas (registros) de "Restaurant20To50"
  criteria = preprocessed_dataset.Restaurant20To50.isna()
  preprocessed_dataset.drop(preprocessed_dataset[criteria].index, inplace=True)

  #Transformamos a números los valores (categorías) de las variables categóricas sin considerar los nulos
  preprocessed_dataset.replace(dictionary_of_columns_with_categorical_to_index, inplace=True)
  
  if save_metadata:
    #Guardamos información que consideremos imporante. El formato pickle nos permite guardar objetos de python
    with open("numeric_columns.pkl", "wb") as f:
      pickle.dump(numeric_columns, f)
    
    with open("categorical_columns.pkl", "wb") as f:
      pickle.dump(categorical_columns, f)
    
    with open("dictionary_of_columns_with_index_to_categorical.pkl", "wb") as f:
      pickle.dump(dictionary_of_columns_with_index_to_categorical, f)
    
    with open("dictionary_of_columns_with_categorical_to_index.pkl", "wb") as f:
      pickle.dump(dictionary_of_columns_with_categorical_to_index, f)
  
  return preprocessed_dataset

In [41]:
# Se preprocesan los datos
preprocessed_dataset = preprocess_dataset(dataset)
preprocessed_dataset.shape
print(type(preprocessed_dataset))

<class 'pandas.core.frame.DataFrame'>


In [42]:
preprocessed_dataset.head()

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,has_children,education,occupation,income,Bar,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
22,1,0,2,55,2,4,0,1,0,2,0,1,0,5,4,3,1,1,3,0,0,0,1,1
23,1,1,2,80,0,2,1,1,0,2,0,1,0,5,4,3,1,1,3,0,0,0,1,0
24,1,1,2,80,0,0,0,1,0,2,0,1,0,5,4,3,1,1,3,0,0,0,1,1
25,1,1,2,80,0,1,1,1,0,2,0,1,0,5,4,3,1,1,3,1,0,0,1,0
26,1,1,2,80,2,2,0,1,0,2,0,1,0,5,4,3,1,1,3,0,0,0,1,0


In [43]:
#Guardamos nuestro dataset pre-procesado en formato pickle
preprocessed_dataset.to_pickle("preprocessed_dataset_titanic_train.pkl")

#Cargamos los datos previamente guardados
with open("numeric_columns.pkl", "rb") as f:
    numeric_columns = pickle.load(f)

with open("categorical_columns.pkl", "rb") as f:
  categorical_columns = pickle.load(f)

with open("dictionary_of_columns_with_index_to_categorical.pkl", "rb") as f:
  dictionary_of_columns_with_index_to_categorical = pickle.load(f)

with open("dictionary_of_columns_with_categorical_to_index.pkl", "rb") as f:
  dictionary_of_columns_with_categorical_to_index = pickle.load(f)

preprocessed_dataset = pd.read_pickle("preprocessed_dataset_titanic_train.pkl")
print(type(preprocessed_dataset))

<class 'pandas.core.frame.DataFrame'>


In [44]:
numeric_columns

['temperature',
 'has_children',
 'toCoupon_GEQ15min',
 'toCoupon_GEQ25min',
 'direction_same',
 'direction_opp',
 'Y']

In [45]:
categorical_columns

['destination',
 'passanger',
 'weather',
 'time',
 'coupon',
 'expiration',
 'gender',
 'age',
 'maritalStatus',
 'education',
 'occupation',
 'income',
 'Bar',
 'CoffeeHouse',
 'CarryAway',
 'RestaurantLessThan20',
 'Restaurant20To50']

In [46]:
# Porcetaje de filas (registros) vacias por caracteristica 
preprocessed_dataset.isna().sum() / len(preprocessed_dataset)

destination             0.0
passanger               0.0
weather                 0.0
temperature             0.0
time                    0.0
coupon                  0.0
expiration              0.0
gender                  0.0
age                     0.0
maritalStatus           0.0
has_children            0.0
education               0.0
occupation              0.0
income                  0.0
Bar                     0.0
CoffeeHouse             0.0
CarryAway               0.0
RestaurantLessThan20    0.0
Restaurant20To50        0.0
toCoupon_GEQ15min       0.0
toCoupon_GEQ25min       0.0
direction_same          0.0
direction_opp           0.0
Y                       0.0
dtype: float64

# Seleccion de caracteristicas: Analisis de componentes principales

In [47]:
preprocessed_dataset = preprocessed_dataset.reset_index(drop=True)
preprocessed_dataset.dtypes
preprocessed_dataset.head(10)

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,has_children,education,occupation,income,Bar,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,1,0,2,55,2,4,0,1,0,2,0,1,0,5,4,3,1,1,3,0,0,0,1,1
1,1,1,2,80,0,2,1,1,0,2,0,1,0,5,4,3,1,1,3,0,0,0,1,0
2,1,1,2,80,0,0,0,1,0,2,0,1,0,5,4,3,1,1,3,0,0,0,1,1
3,1,1,2,80,0,1,1,1,0,2,0,1,0,5,4,3,1,1,3,1,0,0,1,0
4,1,1,2,80,2,2,0,1,0,2,0,1,0,5,4,3,1,1,3,0,0,0,1,0
5,1,1,2,80,2,2,1,1,0,2,0,1,0,5,4,3,1,1,3,1,0,0,1,0
6,1,1,2,80,2,2,0,1,0,2,0,1,0,5,4,3,1,1,3,1,0,0,1,0
7,1,1,2,80,2,4,0,1,0,2,0,1,0,5,4,3,1,1,3,1,0,0,1,1
8,1,1,2,80,3,2,1,1,0,2,0,1,0,5,4,3,1,1,3,0,0,0,1,0
9,1,1,2,80,3,4,1,1,0,2,0,1,0,5,4,3,1,1,3,1,0,0,1,1


In [48]:
X = preprocessed_dataset.drop("Y", axis=1)
Y = preprocessed_dataset["Y"]

In [49]:
# Se realiza el escalamiento de la data para reducir efectos de mayor relevancia de algunas variables sobre otras
from sklearn.preprocessing import scale
dataX = scale(X)
print(type(dataX))

<class 'numpy.ndarray'>


In [50]:
# Se vuelve al formato de pandas
x_pd = pd.DataFrame(dataX, index=X.index, columns=X.columns)
y_pd = Y
print(type(dataset))

<class 'pandas.core.frame.DataFrame'>


In [51]:
# Se convierte a un dataframe de pandas.
#X_rel = pd.DataFrame(dataX, index=X.index, columns=X.columns)
#Y_rel = pd.DataFrame(dataY, index=Y.index, columns=Y.columns)

In [52]:
#Mediante el método "train_test_split" usaremos el 20% de la data para probar el modelo. El parámetro "random state" nos sirve para
#poder replicar la misma separación
x_train, x_test, y_train, y_test = train_test_split(x_pd, y_pd, test_size=0.2, random_state=42)

Fuentes: 
* RandomForest: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
* SVM: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
* XGBoost: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
* GridSearchCV para tuneo de parametros: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
* Recursive Feature Elimination(RFECV): https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html
* Combinacion de tuneo de parametros y eliminacion recursiva de caracteristicas: https://stackoverflow.com/questions/32208546/combining-recursive-feature-elimination-and-grid-search-in-scikit-learn

In [63]:
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

# Se utiliza una grilla para evalular tunear los mejores parametros
def train_cross_validation(method, x_train, y_train, x_test, y_test):
  if (method == "RandForest"):
    clasif = RandomForestClassifier()
    param_grid = {'n_estimators': [50, 100, 200],
               'max_depth': [50, 100],
               'min_samples_split': [8, 10],
               'min_samples_leaf': [3, 4],
               'bootstrap': [True]}
  elif (method == "SVM"):
    clasif = SVC()
    param_grid = {'kernel': ['linear', 'rbf'],
                  'C': [0.01, 0.1, 1],
                  'gamma': [ 0.01, 0.1, 1]}
  elif (method == "XGB"):
    clasif = GradientBoostingClassifier()
    param_grid = {'n_estimators': [100, 200],
                  'learning_rate': [0.1, 0.5],
                  'max_depth': [3, 4],
                  'max_features': ['auto','sqrt','log2']}
  elif (method == "GNB"):
    clasif = GaussianNB()
    param_grid = {'var_smoothing': np.logspace(0,-9, num=100)}

  GridClasif = GridSearchCV(estimator = clasif, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)
  GridClasif.fit(x_train, y_train)
  
  best_estim = GridClasif.best_estimator_

  y_pred = best_estim.predict(x_test)
  
  # Se evalua el score unicamente con los parametros tuneados y sin reduccion de caracteristicas
  acc_tun = metrics.accuracy_score(y_test, y_pred)

  # Se realiza la reduccion de caracteristicas por wrapping
  rfecv = RFECV(estimator=best_estim, step=1, cv=3, scoring='accuracy', min_features_to_select=1)
  rfecv.fit(x_train, y_train)

  # Se evalua el score luego de realizar la reduccion de caracteristicas
  y_pred = rfecv.predict(x_test)
  acc_red = metrics.accuracy_score(y_test, y_pred)

  return acc_tun, acc_red

In [ ]:
[tun, red] = train_cross_validation("RandForest", x_train.to_numpy(), y_train.to_numpy(), x_test.to_numpy(), y_test.to_numpy())
print(tun)
print(red)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   28.0s
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:   36.2s finished


0.7402164862614488
0.746044962531224


In [ ]:
[tun, red] = train_cross_validation("SVM", x_train.to_numpy(), y_train.to_numpy(), x_test.to_numpy(), y_test.to_numpy())
print(tun)
print(red)

Fitting 3 folds for each of 18 candidates, totalling 54 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed:  2.2min finished


0.6915070774354705
0.7431307243963364


In [60]:
[tun, red] = train_cross_validation("XGB", x_train.to_numpy(), y_train.to_numpy(), x_test.to_numpy(), y_test.to_numpy())
print(tun)
print(red)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   25.2s
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:   48.2s finished


0.7402164862614488
0.7447960033305578


In [64]:
[tun, red] = train_cross_validation("GNB", x_train.to_numpy(), y_train.to_numpy(), x_test.to_numpy(), y_test.to_numpy())
print(tun)
print(red)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 246 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    2.5s finished


RuntimeError: ignored

Naive Bayes

Fuente: https://scikit-learn.org/stable/modules/naive_bayes.html

In [ ]:
#Entrenamos el Gaussian Naive Bayes de clasificación
model_GNB = GaussianNB()

model_GNB.fit(x_train, y_train)

In [ ]:
model_GNB.score(x_test, y_test)

In [ ]:
#Entrenamos el Multinomial Naive Bayes de clasificación
model_MNB = MultinomialNB()

model_MNB.fit(x_train, y_train)

In [ ]:
model_MNB.score(x_test, y_test)

In [ ]:
#Entrenamos el Bernoulli Naive Bayes de clasificación
model_BNB = BernoulliNB()

model_BNB.fit(x_train, y_train)

In [ ]:
model_BNB.score(x_test, y_test)

SVM

Fuente: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
svm = make_pipeline(StandardScaler(), SVC(gamma='auto'))
svm.fit(x_test, y_test)

In [ ]:
svm.score(x_test,y_test)

XGBoost

Fuente: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

In [ ]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
xgb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.5, max_depth=2, random_state=0).fit(x_train, y_train)

In [ ]:
xgb.score(x_test, y_test)